# Overview

In this notebook, we illustrate some commonly used augmentations for images in various libraries, and perform inference with both TensorFlow and PyTorch pipelines.

A key augmentation that we will like to illustrate is the insect augmentation, or rather an augmentation that performs an overlay of small artifacts onto the images with a certain probability. A variant first surfaced in the Melanoma Detection competition recently, created by Roman. I've also adapted it in a recent Global Wheat Detection competition. 

## Version Notes

- version 1 to 13: uses images from Casava leaf classification competition
- version 14 onwards: uses images from Plant Pathology 2021, Herbarium 2021, and iWildcam 2021 competitions

In [ ]:
#!pip install imutils
import sys
sys.path.append('../input/imutils/imutils-0.5.3')

# Imports

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import transforms
import tensorflow as tf
import albumentations as A
import imgaug.augmenters as iaa
from imgaug import parameters as iap

# Configs

In [ ]:
Cassava = False
FGVC8_HERB = False # herbarium 2021
FGVC8_CAM = False # iwildcam2021
FGVC8_PLANT = True # plant pathology 2021

if Cassava:
    DIR = "../input/cassava-leaf-disease-classification/train_images"
    image_path = f'{DIR}/100042118.jpg'
if FGVC8_PLANT:
    DIR = "../input/plant-pathology-2021-fgvc8/train_images"
    image_path = f'{DIR}/800113bb65efe69e.jpg'
if FGVC8_HERB:
    DIR = "../input/herbarium-2021-fgvc8/train/images/000/00"
    image_path = f'{DIR}/1360648.jpg'
if FGVC8_CAM:
    DIR = "../input/iwildcam2021-fgvc8/train"
    image_path = f'{DIR}/86760c00-21bc-11ea-a13a-137349068a90.jpg'

# Read Sample Image without Augmentations

In [ ]:
chosen_image = cv2.imread(image_path)
plt.imshow(chosen_image)

# Albumentations Augmentations

- Albumentations part adapted from my good friend Hongnan's notebbok in the Global Wheat Detection competition (https://www.kaggle.com/reighns/augmentations-data-cleaning-and-bounding-boxes#Bounding-Boxes-with-Albumentations)
- Added more augmentations which may be useful
- Added TensorFlow and Torchvision versions of the augmentations

In [ ]:
albumentation_list = [A.RandomSunFlare(p=1), 
                      A.RandomFog(p=1), 
                      A.RandomBrightness(p=1),
                      A.RandomCrop(p=1,height = 512, width = 512), 
                      A.Rotate(p=1, limit=90),
                      A.RGBShift(p=1), 
                      A.RandomSnow(p=1),
                      A.HorizontalFlip(p=1), 
                      A.VerticalFlip(p=1), 
                      A.RandomContrast(limit = 0.5,p = 1),
                      A.HueSaturationValue(p=1,hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=50),
                      A.Cutout(p=1),
                      A.Transpose(p=1), 
                      A.JpegCompression(p=1),
                      A.CoarseDropout(p=1),
                      A.IAAAdditiveGaussianNoise(loc=0, scale=(2.5500000000000003, 12.75), per_channel=False, p=1),
                      A.IAAAffine(scale=1.0, translate_percent=None, translate_px=None, rotate=0.0, shear=0.0, order=1, cval=0, mode='reflect', p=1),
                      A.IAAAffine(rotate=90., p=1),
                      A.IAAAffine(rotate=180., p=1)]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in albumentation_list:
    img = aug_type(image = chosen_image)['image']
    img_matrix_list.append(img)

img_matrix_list.insert(0,chosen_image)    

titles_list = ["Original","RandomSunFlare","RandomFog","RandomBrightness",
               "RandomCrop","Rotate", "RGBShift", "RandomSnow","HorizontalFlip", "VerticalFlip", "RandomContrast","HSV",
               "Cutout","Transpose","JpegCompression","CoarseDropout","IAAAdditiveGaussianNoise","IAAAffine","IAAAffineRotate90","IAAAffineRotate180"]

def plot_multiple_img(img_matrix_list, title_list, ncols, nrows=5,  main_title=""):
    fig, myaxes = plt.subplots(figsize=(20, 15), nrows=nrows, ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize = 30)
    fig.subplots_adjust(wspace=0.3)
    fig.subplots_adjust(hspace=0.3)
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize=15)
    plt.show()
    
plot_multiple_img(img_matrix_list, titles_list, ncols = 4,main_title="Different Types of Augmentations with Albumentations")

# OpenCV-Based Custom Augmentations

# Insect (Bee) Augmentation

- Originally, I implemented this for the recent Global Wheat Detection competition at https://www.kaggle.com/khoongweihao/insect-augmentation-with-efficientdet-d6, inspired by Roman's hair augmentation from the recent Melanoma competition
- Motivations for this approach:
    - the intuition was to mimic as close to as possible, actual scenarios in the wild where insects are present and at times visible in images similar to those in this and similar competitions
    - insects can be found roaming on leaves in reality
    - bees were used as an example. Other small artifacts such as raindrops overlayed with the images or other insect species should suffice. Main idea is to mimic what we can observe in reality, in the form of augmentations

In [ ]:
def insect_augmentation(image, n_insects=2, dark_insect=False, p=0.5, insects_folder='../input/bee-augmentation'):
    aug_prob = random.random()
    if aug_prob < p:
        height, width, _ = image.shape  # target image width and height
        insects_images = [im for im in os.listdir(insects_folder) if 'bee' in im]
        img_shape = image.shape

        for _ in range(n_insects):
            insect = cv2.cvtColor(cv2.imread(os.path.join(insects_folder, random.choice(insects_images))), cv2.COLOR_BGR2RGB)
            insect = cv2.flip(insect, random.choice([-1, 0, 1]))
            insect = cv2.rotate(insect, random.choice([0, 1, 2]))
            insect = cv2.resize(insect, (width, height))

            h_height, h_width, _ = insect.shape  # insect image width and height
            roi_ho = random.randint(0, image.shape[0] - insect.shape[0])
            roi_wo = random.randint(0, image.shape[1] - insect.shape[1])
            roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask 
            img2gray = cv2.cvtColor(insect, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            #mask_inv = cv2.cvtColor(cv2.bitwise_not(mask),cv2.COLOR_BGR2GRAY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of insect in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of insect from insect image.
            if dark_insect:
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                insect_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
            else:
                insect_fg = cv2.bitwise_and(insect, insect, mask=mask)

            # Put insect in ROI and modify the target image
            dst = cv2.add(img_bg, insect_fg, dtype=cv2.CV_64F)

            image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

    return image

### Original Bees

In [ ]:
chosen_image = cv2.imread(image_path)
aug_image = insect_augmentation(chosen_image, n_insects=2, dark_insect=False, p=1.0)
plt.imshow(aug_image)

### Dark Bees

In [ ]:
chosen_image = cv2.imread(image_path)
aug_image = insect_augmentation(chosen_image, n_insects=2, dark_insect=True, p=1.0)
plt.imshow(aug_image)

# `imgaug` Based Augmentations

In [ ]:
ia_trans_list = [iaa.blend.BlendAlpha(factor=(0.2, 0.8),
                                      foreground=iaa.Affine(rotate=(-30, 30)),
                                      per_channel=True),
                 iaa.Fliplr(1.),
                 iaa.Flipud(1.),
                 iaa.SimplexNoiseAlpha(iaa.Multiply(iap.Choice([0.5, 1.5]), per_channel=True)),
                 iaa.Crop(percent=(0., 0.3)),
                ]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in ia_trans_list:
    # convert to tensor
    chosen_image = cv2.imread(image_path)
    iaa_seq = iaa.Sequential([aug_type])
    trans_img = iaa_seq.augment_images(chosen_image)
    img_matrix_list.append(trans_img)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","Ghost Aug","Flip Left Right","Flip Up Down","SimplexNoiseAlpha", "Crop"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=2, main_title="Different Types of Augmentations with Albumentations")

# PyTorch-Based Augmentations (Torchvision)

In [ ]:
torch_trans_list = [transforms.CenterCrop((178, 178)),
                    transforms.Resize(128),
                    transforms.RandomRotation(45),
                    transforms.RandomAffine(35),
                    transforms.RandomCrop(128),
                    transforms.RandomHorizontalFlip(p=1),
                    transforms.RandomPerspective(p=1),
                    transforms.RandomVerticalFlip(p=1)]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in torch_trans_list:
    # convert to tensor
    chosen_image = cv2.imread(image_path)
    chosen_tensor = transforms.Compose([transforms.ToTensor()])(chosen_image)
    chosen_tensor = transforms.Compose([aug_type])(chosen_tensor)
    trans_img = transforms.ToPILImage()(chosen_tensor)
    img_matrix_list.append(trans_img)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","CenterCrop","Resize","RandomRotation","RandomAffine","RandomCrop","RandomHorizontalFlip","RandomPerspective",
               "RandomVerticalFlip"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=3, main_title="Different Types of Augmentations with Albumentations")

# TensorFlow-Based Augmentations

In [ ]:
chosen_image = cv2.imread(image_path)

tf_trans_list = [
    tf.image.rot90(chosen_image, k=1), # 90 degrees counter-clockwise
    tf.image.rot90(chosen_image, k=2), # 180 degrees counter-clockwise
    tf.image.rot90(chosen_image, k=3), # 270 degrees counter-clockwise
    tf.image.random_brightness(chosen_image, 0.5), 
    tf.image.random_contrast(chosen_image, 0.2, 0.5), 
    tf.image.random_flip_left_right(chosen_image, seed=42),
    tf.image.random_flip_up_down(chosen_image, seed=42),
    tf.image.random_hue(chosen_image, 0.5),
    tf.image.random_jpeg_quality(chosen_image, 35, 50), 
    tf.image.random_saturation(chosen_image, 5, 10), 
    tf.image.transpose(chosen_image),
]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_image in tf_trans_list:
    img_matrix_list.append(aug_image)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","Rotate90","Rotate180","Rotate270","RandomBrightness","RandomContrast","RandomLeftRightFlip","RandomUpDownFlip",
               "RandomHue","RandomJPEGQuality","RandomSaturation","Transpose"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=4, main_title="Different Types of Augmentations with Albumentations")

# Model Pipeline with Insect (Bee) Augmentation

## TensorFlow-Keras

- modified from @dimitreoliveira's inference notebook to perform inference for weights obtained with insect augmentation: https://www.kaggle.com/dimitreoliveira/cassava-leaf-disease-tpu-v2-pods-inference

In [ ]:
!pip install --quiet /kaggle/input/kerasapplications
!pip install --quiet /kaggle/input/efficientnet-git

In [ ]:
import math, os, re, warnings, random, glob
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential, Model
import efficientnet.tfkeras as efn

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

BATCH_SIZE = 16 * REPLICAS
HEIGHT = 512
WIDTH = 512 
CHANNELS = 3
N_CLASSES = 5
TTA_STEPS = 8 # Do TTA if > 0 

def data_augment(image, label):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
            
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.7)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.8)
        else:
            image = tf.image.central_crop(image, central_fraction=.9)
    elif p_crop > .4:
        crop_size = tf.random.uniform([], int(HEIGHT*.8), HEIGHT, dtype=tf.int32)
        image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])
    
    return image, label

def get_name(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    name = parts[-1]
    return name

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    
    return image

def center_crop(image):
    image = tf.reshape(image, [600, 800, CHANNELS]) # Original shape
    
    h, w = image.shape[0], image.shape[1]
    if h > w:
        image = tf.image.crop_to_bounding_box(image, (h - w) // 2, 0, w, w)
    else:
        image = tf.image.crop_to_bounding_box(image, 0, (w - h) // 2, h, h)
        
    image = tf.image.resize(image, [HEIGHT, WIDTH]) # Expected shape
    return image

def resize_image(image, label):
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = tf.reshape(image, [HEIGHT, WIDTH, CHANNELS])
    return image, label

def process_path(file_path):
    name = get_name(file_path)
    img = tf.io.read_file(file_path)
    img = decode_image(img)
    return img, name

def get_dataset(files_path, shuffled=False, tta=False, extension='jpg'):
    dataset = tf.data.Dataset.list_files(f'{files_path}*{extension}', shuffle=shuffled)
    dataset = dataset.map(process_path, num_parallel_calls=AUTO)
    if tta:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.map(resize_image, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

database_base_path = '/kaggle/input/cassava-leaf-disease-classification/'
submission = pd.read_csv(f'{database_base_path}sample_submission.csv')
display(submission.head())

TEST_FILENAMES = tf.io.gfile.glob(f'{database_base_path}test_tfrecords/ld_test*.tfrec')
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print(f'GCS: test: {NUM_TEST_IMAGES}')

model_path_list = glob.glob('../input/casava-efficientnet-tpu-weights/*.h5')
model_path_list.sort()

print('Models to predict:')
print(*model_path_list, sep='\n')

def model_fn(input_shape, N_CLASSES):
    inputs = L.Input(shape=input_shape, name='input_image')
    base_model = efn.EfficientNetB4(input_tensor=inputs, 
                                    include_top=False, 
                                    weights=None, 
                                    pooling='avg')

    x = L.Dropout(.5)(base_model.output)
    output = L.Dense(N_CLASSES, activation='softmax', name='output')(x)
    model = Model(inputs=inputs, outputs=output)

    return model

with strategy.scope():
    model = model_fn((None, None, CHANNELS), N_CLASSES)
    
files_path = f'{database_base_path}test_images/'
test_size = len(os.listdir(files_path))
test_preds = np.zeros((test_size, N_CLASSES))


for model_path in model_path_list:
    print(model_path)
    K.clear_session()
    model.load_weights(model_path)

    if TTA_STEPS > 0:
        test_ds = get_dataset(files_path, tta=True).repeat()
        ct_steps = TTA_STEPS * ((test_size/BATCH_SIZE) + 1)
        preds = model.predict(test_ds, steps=ct_steps, verbose=1)[:(test_size * TTA_STEPS)]
        preds = np.mean(preds.reshape(test_size, TTA_STEPS, N_CLASSES, order='F'), axis=1)
        test_preds += preds / len(model_path_list)
    else:
        test_ds = get_dataset(files_path, tta=False)
        x_test = test_ds.map(lambda image, image_name: image)
        test_preds += model.predict(x_test) / len(model_path_list)
    
test_preds = np.argmax(test_preds, axis=-1)
test_names_ds = get_dataset(files_path)
image_names = [img_name.numpy().decode('utf-8') for img, img_name in iter(test_names_ds.unbatch())]

submission = pd.DataFrame({'image_id': image_names, 'label': test_preds})
submission.to_csv('submission.csv', index=False)

# Useful References

- https://www.tensorflow.org/api_docs/python/tf/image/
- https://albumentations.ai/docs/api_reference/augmentations/transforms/
- https://pytorch.org/docs/stable/torchvision/transforms.html

# Some Remarks

- the needle augmentation implementation still remains buggy with the TensorFlow-Keras pipeline
- in Torch, it seems to work just fine
- after the competition has concluded and after reviewing private and public LB scores, it does seem that using insect augmentations led to a closer private and public LB score relationship! But sadly no medal this time round.. Sobs...

# Notable Cases Where Insect Augmentation Works

- improved CV scores (LB wasn't observed as final submissions) when applied to wheat data in the recent Global Wheat Detection competition
- (replacing bees with xray needles) improved CV and LB scores observed when applied to medical image use cases, e.g. in RANZCR competition

## All the best Kagglers!